# Example: Impact Fund Screening
In this example we score potential land acquisitions in batch. The workflow shows how to combine land-cover metrics, B-Score calculation and occurrence density.

In [18]:
from pathlib import Path
import pandas as pd
from verdesat.services.bscore import compute_bscores
from verdesat.biodiv.gbif_validator import OccurrenceService, plot_score_vs_density

In [19]:
geojson_path = "examples/reforestation_plots.geojson"

In [22]:
# Load AOIs from GeoJSON
from verdesat.geo.aoi import AOI
projects_path = geojson_path
projects = AOI.from_geojson(projects_path, id_col='ID2')

In [23]:
df_scores = compute_bscores(projects_path, year=2021, output='examples/bscore.csv')
df_scores['id'] = df_scores['id'].astype(int)

2025-07-28 19:29:11 [INFO] verdesat.services.bscore – Loading AOIs from examples/reforestation_plots.geojson
2025-07-28 19:29:11 [INFO] verdesat.services.bscore – Loading landcover image projects/sat-io/open-datasets/landcover/ESRI_Global-LULC_10m_TS
2025-07-28 19:29:13 [INFO] verdesat.services.bscore – ✔ Converted to COG: /var/folders/bc/p78br9810cd0yyjhgq_sz2kc0000gn/T/tmpeuyg1925/LANDCOVER_1_2021.tiff
2025-07-28 19:29:13 [INFO] verdesat.services.bscore – Wrote landcover raster to /var/folders/bc/p78br9810cd0yyjhgq_sz2kc0000gn/T/tmpeuyg1925/LANDCOVER_1_2021.tiff
2025-07-28 19:29:13 [INFO] verdesat.services.bscore – Loading landcover image projects/sat-io/open-datasets/landcover/ESRI_Global-LULC_10m_TS
2025-07-28 19:29:17 [INFO] verdesat.services.bscore – ✔ Converted to COG: /var/folders/bc/p78br9810cd0yyjhgq_sz2kc0000gn/T/tmppkfsi4tg/LANDCOVER_2_2021.tiff
2025-07-28 19:29:17 [INFO] verdesat.services.bscore – Wrote landcover raster to /var/folders/bc/p78br9810cd0yyjhgq_sz2kc0000gn/T/t

In [24]:
occ = OccurrenceService()
import geopandas as gpd
densities = []
for aoi in projects:
    aoi_gdf = gpd.GeoDataFrame({'geometry': [aoi.geometry]}, crs='EPSG:4326')
    gdf = occ.fetch_occurrences(aoi.geometry)
    area_km2 = aoi_gdf.to_crs(epsg=6933).area.iloc[0] / 1e6
    densities.append(occ.occurrence_density_km2(gdf, area_km2))
plot_score_vs_density(df_scores['bscore'].tolist(), densities, 'examples/score_vs_density.png')

2025-07-28 19:29:36 [INFO] verdesat.biodiv.gbif_validator – Fetching occurrences since 2000 for bbox (-92.53968503301898, 16.790645159322896, -92.5365480721544, 16.792274)
2025-07-28 19:29:36 [INFO] verdesat.biodiv.gbif_validator – Fetched 12 GBIF records
2025-07-28 19:29:36 [INFO] verdesat.biodiv.gbif_validator – EBIRD_TOKEN not set; skipping eBird fallback
2025-07-28 19:29:36 [INFO] verdesat.biodiv.gbif_validator – Querying iNaturalist within bbox (-92.53968503301898, 16.790645159322896, -92.5365480721544, 16.792274)
2025-07-28 19:29:36 [INFO] pyinaturalist – Request:
GET https://api.inaturalist.org/v1/observations?d1=2000-01-01T00%3A00%3A00%2B01%3A00&nelat=16.792274&nelng=-92.5365480721544&swlat=16.790645159322896&swlng=-92.53968503301898
User-Agent: python-requests/2.32.4 pyinaturalist/0.20.1
Accept-Encoding: gzip, deflate, zstd
Accept: application/json
Connection: keep-alive

2025-07-28 19:29:37 [INFO] verdesat.biodiv.gbif_validator – Fetched 2 iNaturalist records
2025-07-28 19:29

The scatter plot compares biodiversity score with citizen-science occurrence density for each polygon.